In [ ]:
import numpy as np
import yaml
from hipe4ml.model_handler import ModelHandler
import ROOT
from ROOT import RooFit, RooRealVar, RooArgusBG, RooGaussian, RooDataSet, RooArgList, RooCategory, RooSimultaneous, RooAddPdf
ROOT.gSystem.Load('../pdfLib/libRooATan.dylib')
import math
import pickle
import os
import sys
sys.path.append('../include')
import myHeader as myH
import para as para
import fRead as fRead
import emTool as emTool
from Data import analysisData
from copy import deepcopy

#ROOT.Math.IntegratorOneDimOptions.SetDefaultRelTolerance(1.E-16)
ROOT.Math.IntegratorOneDimOptions.SetDefaultIntegrator("GaussLegendre")
#“1-dim integrators”: “Gauss”, “GaussLegendre”, “Adaptive”, “AdaptiveSingular” “NonAdaptive”
myH.gStyleInit()

In [ ]:
# Set paramters
CENT_BIN_LIST = para.CENT_BIN_LIST
PT_BIN_LIST = para.PT_BIN_LIST
# PT_BIN_LIST = para.PT_BIN_LIST_NEW
MASS_BIN = para.MASS_BIN
MODEL_Eff_LIST = para.MODEL_Eff_LIST

sigpdf = ["dscb"]
# bkgpdf = ["argus", "Landau", "pol5", "dscb"]
# bkgpdf = ["argus", "pol3", "dscb"]
bkgpdf = ["pol3"]

ifTellMatter = False
dataType = "24skimmed_reduced"

ifUseML = True

if len(sigpdf) == 1 and len(bkgpdf) == 1:
    savePrefix = f'Plot/{sigpdf[0]}_{bkgpdf[0]}/'
else:
    savePrefix = "Plot/"
if not os.path.exists(savePrefix):
    os.makedirs(savePrefix)
# ModelPath = "../MLAna/Model/24skimmed"
# ModelPath = "../MLAna_LikeSign/Model/24skimmed"
ModelPath = "../MLAna_LikeSign_LooseCut/Model/24skimmed"
# ModelPath = "../MLAna_LikeSign_LooseCut_2_2.6_5/Model/24skimmed"

score_eff_arrays_dict = pickle.load(
    open(ModelPath + "/file_score_eff_dict", "rb"))

config_file_path = fRead.getConfigPath(dataType)
config_file = open(config_file_path, 'r')
config = yaml.full_load(config_file)

BR_3body = 0.4

In [ ]:
############### Readin dataset and model ############### 
DataTH = fRead.getDataTH("24skimmed")
MCTH = fRead.getMCTH(dataType)
BkgTH_mixedDeuteron = fRead.getBkgTH("24skimmed_newReduced", "mixed_deuteron", DataTH, period="24amao")
BkgTH_mixed_proton = fRead.getBkgTH("24skimmed_newReduced", "mixed_proton", DataTH, period="24amao")

DataPDRaw = DataTH.get_data_frame()
MCPDRaw = MCTH.get_data_frame()
BkgPDRaw_mixed_deuteron = BkgTH_mixedDeuteron.get_data_frame()
BkgPDRaw_mixed_proton = BkgTH_mixed_proton.get_data_frame()

myH.calNewElements(DataPDRaw)
myH.calNewElements(BkgPDRaw_mixed_deuteron)
myH.calNewElements(MCPDRaw)
myH.calNewElements(BkgPDRaw_mixed_proton)

DataTH.set_data_frame(DataPDRaw)
BkgTH_mixedDeuteron.set_data_frame(BkgPDRaw_mixed_deuteron)
BkgTH_mixed_proton.set_data_frame(BkgPDRaw_mixed_deuteron)
MCTH.set_data_frame(MCPDRaw)

print("Count of Raw Data: ", len(DataPDRaw))
print("Count of Raw bkg_mixed_deuteron: ", len(BkgPDRaw_mixed_deuteron))
print("Count of Raw bkg_mixed_proton: ", len(BkgPDRaw_mixed_proton))

if len(CENT_BIN_LIST) != 1:
    raise ValueError("CENT_BIN_LIST should have only one element") # Now we only consider one centrality bin

model_hdl = myH.createEmptyList( [len(CENT_BIN_LIST)] )
for icent, centbin in enumerate(CENT_BIN_LIST):
    for ptbin in PT_BIN_LIST[icent]:
        modelfile_name = ModelPath + '/Model' + "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
        modelReadin = ModelHandler()
        modelReadin.load_model_handler(modelfile_name)
        model_hdl[icent].append(deepcopy(modelReadin))

In [ ]:
############### Reweight pt shape ###############
pt_spectrum = fRead.getHypertritonPtShape(dataType)
myH.apply_pt_rejection(MCPDRaw, [pt_spectrum], [[-100, 999]], PT_BIN_LIST) # centrality unavailable
MCPDRaw = MCPDRaw.query("rej == False and fSurvivedEventSelection == True")
# FIXME: The raw pt distribution of mixing background is not flat! Reweight not meaningful
# myH.apply_pt_rejection(BkgPDRaw_mixed_deuteron, [pt_spectrum], [[-100, 999]], PT_BIN_LIST, ptcolumn="fPt") # centrality unavailable
# BkgPDRaw_mixed_deuteron = BkgPDRaw_mixed_deuteron.query("rej == False")

print("After pT reweight")
print("MC dataset:",len(MCPDRaw))
print("Generated MC hypertirton within |y| < 0.5:", len(MCPDRaw.query("fIsSignal == 1 and abs(fGenRapidity) < 0.5")))
print("Reconstructed MC hypertirton within 2 <= pT < 5:", len(MCPDRaw.query("fIsSignal == 1 and fPt >= 2 and fPt < 5")))

In [ ]:
############### Apply precut and add model prediction to data ###############
# print(DataPDRaw.columns)

# minCosPA = 0.995
# maxDCADaughters = 0.1
# preCut =   f"fCt >= 2 and fCt < 40 and \
#             fCosPA > {minCosPA} and \
#             abs(fTPCNSigmaProton) < 3 and abs(fTPCNSigmaBachelor) < 3 and \
#             fDCADaughters < {maxDCADaughters} and \
#             fDCAPionToPV > 0.05"

if "LooseCut" in ModelPath:
    preCut = myH.convert_sel_to_string(config['MLLoosePreSelection'])
else:
    preCut = myH.convert_sel_to_string(config['MLPreSelection'])
print("ML precut:", preCut)

data = analysisData(DataPDRaw, preCut, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN)
data.addModelPrediction(model_hdl)
mcdata = analysisData(MCPDRaw, preCut, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN) # cut for pt intervals include fPt > 0
mcdata.addModelPrediction(model_hdl)
bkg_mixed_deuteron = analysisData(BkgPDRaw_mixed_deuteron, preCut, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN)
bkg_mixed_deuteron.addModelPrediction(model_hdl)
bkg_mixed_proton = analysisData(BkgPDRaw_mixed_proton, preCut, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN)
bkg_mixed_proton.addModelPrediction(model_hdl)

In [ ]:
############### Fit MC invariant mass distribution to extract the shape of signal ###############
MCFitpara = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0])] ) # only for DSCB
with ROOT.TFile(savePrefix + "MCfit.root", "recreate") as outfile:
    for icent, centbin in enumerate(CENT_BIN_LIST):
        for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
            (_, __, paras) = mcdata.invMFit(icent, ipt, "DSCB", bkgpdf="none", isMC=True, ifDrawStats=False, outfile = outfile)
            MCFitpara[icent][ipt].append(paras)

In [ ]:
kOrangeC = ROOT.kOrange+7
kBlueC = ROOT.kBlue+2
kGreenC = ROOT.kSpring+3

def simultaneousFit(df_se, df_mixingDeuteron, df_mixingProton, mcparas, nBins, ptlims, lowMassLim = 2.96, highMassLim = 3.04, simBkgFit = True, title = '', signalPdf = 'dscb', uncorr_bkgPdf = 'pol1', corr_bkgPdf = 'dscb', df_column = 'fMass'):
  
  signalFitOptions = ['gauss', 'dscb']
  if signalPdf not in signalFitOptions:
    raise ValueError(
      f'Invalid signalPdf option. Expected one of: {signalFitOptions}')
  
  uncorrBkgFitOptions = ['pol0', 'pol1', 'pol2', 'expo']
  if uncorr_bkgPdf not in uncorrBkgFitOptions:
    raise ValueError(
      f'Invalid uncorr_bkgPdf option. Expected one of: {uncorrBkgFitOptions}')
  
  corrBkgFitOptions = ['argus', 'dscb', 'atan', 'landau', 'pol3', 'pol4', 'pol5', 'test']
  if corr_bkgPdf not in corrBkgFitOptions:
    raise ValueError(
      f'Invalid corr_bkgPdf option. Expected one of: {corrBkgFitOptions}')
  
  # bin width for axis title
  bin_width = round((highMassLim - lowMassLim)/nBins, 4)
  
  # Define the mass variable
  x = RooRealVar("x", "Invariant Mass", lowMassLim, highMassLim)
  xShifted =  ROOT.RooFormulaVar("xShifted", "xShifted", "x-3.04", ROOT.RooArgList(x))
  xMirrored = ROOT.RooFormulaVar("xMirrored", "xMirrored", "3.08-x", ROOT.RooArgList(x))

  # Convert pandas data to RooFit dataset
  data_se = myH.ndarray2roo(np.array(df_se['fM']), x, 'data_se')
  data_mixingDeuteron = myH.ndarray2roo(np.array(df_mixingDeuteron[df_column]), x, 'data_mixingDeuteron')
  data_mixingProton = myH.ndarray2roo(np.array(df_mixingProton[df_column]), x, 'data_mixingProton')

  # Category to distinguish datasets
  sample = RooCategory("sample", "Sample Type")
  sample.defineType("mixed_deuteron")
  sample.defineType("mixed_proton")
  sample.defineType("same_event")

  # Combine background data sets
  combData = RooDataSet("combData", "Combined Data", RooArgList(x), RooFit.Index(sample), RooFit.Import("mixed_deuteron", data_mixingDeuteron), RooFit.Import("mixed_proton", data_mixingProton), RooFit.Import("same_event", data_se))

  ############### Define Signal function ###############
  if signalPdf == 'gauss':
    mu = RooRealVar("#mu", "Mean of Gaussian", mcparas[0].getVal(), 2.986, 2.996) # 2.991
    sigma = RooRealVar("#sigma", "Width of Gaussian", 0.0005, 0.0030)
    # fix sigma
    sigma.setVal(mcparas[1].getVal())
    sigma.setConstant(ROOT.kTRUE)
    signal = ROOT.RooGaussian("signal", "signal", x, mu, sigma)
  elif signalPdf == 'dscb':
    mu = ROOT.RooRealVar("#mu", "mu", mcparas[0].getVal(), 2.986, 2.996)
    sigma = ROOT.RooRealVar("#sigma", "sigma", 0.0005, 0.0030)
    a1 = ROOT.RooRealVar("a_{1}", "a1", 0.1, 2)
    n1 = ROOT.RooRealVar("n_{1}", "n1", 0.1, 10)
    a2 = ROOT.RooRealVar("a_{2}", "a2", 0.1, 2)
    n2 = ROOT.RooRealVar("n_{2}", "n2", 0.1, 10)
    # fix the parameters
    sigma.setVal(mcparas[1].getVal())
    a1.setVal(mcparas[2].getVal())
    n1.setVal(mcparas[3].getVal())
    a2.setVal(mcparas[4].getVal())
    n2.setVal(mcparas[5].getVal())
    sigma.setConstant(ROOT.kTRUE)
    a1.setConstant(ROOT.kTRUE)
    n1.setConstant(ROOT.kTRUE)
    a2.setConstant(ROOT.kTRUE)
    n2.setConstant(ROOT.kTRUE)
    signal = ROOT.RooCrystalBall("signal", "signal", x, mu, sigma, a1, n1, a2, n2)

  ############### Define uncorrelated bkg function ###############
  if uncorr_bkgPdf == 'pol0':
    uncorr_bkg = ROOT.RooPolynomial("uncorr_bkg", "uncorr_bkg", x, ROOT.RooArgList())
  elif uncorr_bkgPdf == 'pol1':
    c1 = ROOT.RooRealVar('c1', 'c1', 3, 0, 15.3)
    uncorr_bkg = ROOT.RooPolynomial("uncorr_bkg", "uncorr_bkg", xShifted, ROOT.RooArgList(c1))
  elif uncorr_bkgPdf == 'pol2':
    pol_a = ROOT.RooRealVar('pol_a', 'pol_a', 3, -30, 30) #0-20
    pol_b = ROOT.RooRealVar('pol_b', 'pol_b', 3, -30, 30) #0-20
    uncorr_bkg = ROOT.RooPolynomial("uncorr_bkg", "uncorr_bkg", xShifted, ROOT.RooArgList(pol_a, pol_b))
  elif uncorr_bkgPdf == 'expo':
    c1 = ROOT.RooRealVar('c1', 'c1', -35., -100., -10.)
    uncorr_bkg = ROOT.RooExponential('uncorr_bkg', 'uncorr_bkg', x, c1)

  ############### Define correlated bkg function ###############
  if corr_bkgPdf == "argus":
    if ptlims[0] == 2.0:
      argus_m0 = ROOT.RooRealVar("argus_m0", "argus_m0", 0.095, 0.07, 0.15)
      argus_c = ROOT.RooRealVar("argus_c", "argus_c", -3, -10, -1) #-10 (2, 2.4 binning)
      argus_p = ROOT.RooRealVar("argus_p", "argus_p", 10, 4.5, 50) # 5
    elif ptlims[0] == 3.0:
      argus_m0 = ROOT.RooRealVar("argus_m0", "argus_m0", 0.095, 0.07, 0.15)
      argus_c = ROOT.RooRealVar("argus_c", "argus_c", -3, -10, -1) #-10 (2, 2.4 binning)
      argus_p = ROOT.RooRealVar("argus_p", "argus_p", 10, 4.5, 50) # 5
    corr_bkg = ROOT.RooArgusBG("corr_bkg", "corr_bkg", xMirrored, argus_m0, argus_c, argus_p)
  elif corr_bkgPdf == "dscb":
    bkg_mu = ROOT.RooRealVar("bkg_mu", "bkg_mu", 2.994, 2.986, 3.01)
    bkg_sigma = ROOT.RooRealVar("bkg_sigma", "bkg_sigma", 0.0005, 0.01)
    bkg_a1 = ROOT.RooRealVar("bkg_a_{1}", "bkg_a1", 0.1, 2)
    bkg_n1 = ROOT.RooRealVar("bkg_n_{1}", "bkg_n1", 10, 0.1, 20)
    bkg_a2 = ROOT.RooRealVar("bkg_a_{2}", "bkg_a2", 0.1, 2)
    bkg_n2 = ROOT.RooRealVar("bkg_n_{2}", "bkg_n2", 0.1, 10)
    corr_bkg = ROOT.RooCrystalBall("corr_bkg", "corr_bkg", x, bkg_mu, bkg_sigma, bkg_a1, bkg_n1, bkg_a2, bkg_n2)
  elif corr_bkgPdf == "atan":
    atan_a = ROOT.RooRealVar("atan_a", "atan_a", 0.3, 0.00001, 1000)
    atan_b = ROOT.RooRealVar("atan_b", "atan_b", -1, -1000, 0)
    corr_bkg = ROOT.RooATan("corr_bkg", "corr_bkg", x, atan_a, atan_b)
  elif corr_bkgPdf == "landau":
    landau_m0 = ROOT.RooRealVar("landau_m0", "landau_m0", 2.996, 2.986, 3.01)
    landau_sigma = ROOT.RooRealVar("landau_sigma", "landau_sigma", 0.0005, 0.1)
    corr_bkg = ROOT.RooLandau("corr_bkg", "corr_bkg", x, landau_m0, landau_sigma)
  elif corr_bkgPdf == "pol5":
    bkg_a1 = ROOT.RooRealVar("bkg_a1", "bkg_a1", -1., 1.)
    bkg_a2 = ROOT.RooRealVar("bkg_a2", "bkg_a2", -1., 1.)
    bkg_a3 = ROOT.RooRealVar("bkg_a3", "bkg_a3", -1., 1.)
    bkg_a4 = ROOT.RooRealVar("bkg_a4", "bkg_a4", -1., 1.)
    bkg_a5 = ROOT.RooRealVar("bkg_a5", "bkg_a5", -1., 1.)
    corr_bkg = ROOT.RooChebychev("corr_bkg", "corr_bkg", x, ROOT.RooArgList(bkg_a1, bkg_a2, bkg_a3, bkg_a4, bkg_a5))
  elif corr_bkgPdf == "pol4":
    bkg_a1 = ROOT.RooRealVar("bkg_a1", "bkg_a1", 0.9, -1., 1.)
    bkg_a2 = ROOT.RooRealVar("bkg_a2", "bkg_a2", -0.3,-1., 1.)
    bkg_a3 = ROOT.RooRealVar("bkg_a3", "bkg_a3", -0.3,-1., 1.)
    bkg_a4 = ROOT.RooRealVar("bkg_a4", "bkg_a4", 0.1, -1., 1.)
    corr_bkg = ROOT.RooChebychev("corr_bkg", "corr_bkg", x, ROOT.RooArgList(bkg_a1, bkg_a2, bkg_a3, bkg_a4))
  elif corr_bkgPdf == "pol3":
    bkg_a1 = ROOT.RooRealVar("bkg_a1", "bkg_a1", 0.9, -1., 1.)
    bkg_a2 = ROOT.RooRealVar("bkg_a2", "bkg_a2", -0.3, -1., 1.)
    bkg_a3 = ROOT.RooRealVar("bkg_a3", "bkg_a3", -0.4, -1., 1.)
    corr_bkg = ROOT.RooChebychev("corr_bkg", "corr_bkg", x, ROOT.RooArgList(bkg_a1, bkg_a2, bkg_a3))
  # elif corr_bkgPdf == "test":
  #   bkg1_mu = ROOT.RooRealVar("bkg1_mu", "bkg1_mu", 2.999, 2.997, 3.001)
  #   bkg1_sigma = ROOT.RooRealVar("bkg1_sigma", "bkg1_sigma", 0.0005, 0.01)
  #   bkg1_a1 = ROOT.RooRealVar("bkg1_a_{1}", "bkg1_a1", 0.1, 2)
  #   bkg1_n1 = ROOT.RooRealVar("bkg1_n_{1}", "bkg1_n1", 10, 0.1, 20)
  #   bkg1_a2 = ROOT.RooRealVar("bkg1_a_{2}", "bkg1_a2", 0.1, 2)
  #   bkg1_n2 = ROOT.RooRealVar("bkg1_n_{2}", "bkg1_n2", 0.1, 10)
  #   bkg2_mu = ROOT.RooRealVar("bkg2_mu", "bkg2_mu", 2.994, 2.986, 3.01)
  #   bkg2_sigma = ROOT.RooRealVar("bkg2_sigma", "bkg2_sigma", 0.0005, 0.01)
  #   bkg2_a1 = ROOT.RooRealVar("bkg2_a_{1}", "bkg2_a1", 0.1, 2)
  #   bkg2_n1 = ROOT.RooRealVar("bkg2_n_{1}", "bkg2_n1", 10, 0.1, 20)
  #   bkg2_a2 = ROOT.RooRealVar("bkg2_a_{2}", "bkg2_a2", 0.1, 2)
  #   bkg2_n2 = ROOT.RooRealVar("bkg2_n_{2}", "bkg2_n2", 0.1, 10)
  #   corr_bkg1 = ROOT.RooCrystalBall("corr_bkg1", "corr_bkg1", x, bkg1_mu, bkg1_sigma, bkg1_a1, bkg1_n1, bkg1_a2, bkg1_n2)
  #   corr_bkg2 = ROOT.RooCrystalBall("corr_bkg2", "corr_bkg2", x, bkg2_mu, bkg2_sigma, bkg2_a1, bkg2_n1, bkg2_a2, bkg2_n2)
  #   frac_corr_peak1 = RooRealVar("frac_corr_peak1", "Correlated bkg fraction peak 1", 0.5, 0.0, 1.0)
  #   corr_bkg = ROOT.RooAddPdf("corr_bkg", "corr_bkg", RooArgList(corr_bkg1, corr_bkg2), RooArgList(frac_corr_peak1))

  frac_corr_mixedDeuteron = RooRealVar("frac_corr_mixedDeuteron", "Correlated background fraction while mixing deuterons", 0.01, 0.0, 1.0)
  if simBkgFit == True:
    # Define composite model for mixingDeuteron background
    total_bkg_pdf = RooAddPdf("total_bkg_pdf", "Uncorr. + Corr. Background", RooArgList(corr_bkg, uncorr_bkg), RooArgList(frac_corr_mixedDeuteron))
  elif simBkgFit == False:
    # Define correlated bkg only model for mixingDeuteron background
    total_bkg_pdf = corr_bkg


  # Define composite model for SE background
  frac_corr = RooRealVar("frac_corr", "Correlated background fraction", 0.01, 0.0, 1.0)
  total_sig_bkg_pdf = RooAddPdf("total_sig_bkg_pdf", "Uncorr. + Corr. Background", RooArgList(uncorr_bkg, corr_bkg), RooArgList(frac_corr))

  # Define composite model for SE total
  frac_sig = RooRealVar("frac_sig", "Signal peak fraction", 0.5, 0.0, 1.0)
  total_pdf = RooAddPdf("total_pdf", "Signal + Uncorr. + Corr. Background", RooArgList(signal, total_sig_bkg_pdf), RooArgList(frac_sig))

  # Simultaneous PDF setup
  simPdf = RooSimultaneous("simPdf", "Simultaneous PDF", sample)
  simPdf.addPdf(uncorr_bkg, "mixed_proton")  # Background only fit to mixingProton
  simPdf.addPdf(total_bkg_pdf, "mixed_deuteron")  # Background fit to mixingDeuteron
  simPdf.addPdf(total_pdf, "same_event")  # Signal + Background fit to SE

  # Perform simultaneous fit
  simPdf.fitTo(combData)

  ############################################
  ############### Draw results ###############
  ############################################

  ############### Draw uncorrelated background spectrum with fit ###############
  canvas_bkg_uncorr = ROOT.TCanvas("canvas_bkg_uncorr" + title, "Background uncorrelated", 800, 800)
  frame_bkg_uncorr = x.frame(RooFit.Bins(nBins))  # Apply binning
  data_mixingProton.plotOn(frame_bkg_uncorr, RooFit.Binning(nBins))
  uncorr_bkg.plotOn(frame_bkg_uncorr, RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(kOrangeC))
  frame_bkg_uncorr.GetYaxis().SetTitle(f'Counts / ({bin_width}' + ' GeV/#it{c}^{2})')
  frame_bkg_uncorr.GetXaxis().SetTitle('#it{M}(p+#pi+d) (GeV/#it{c}^{2})')
  frame_bkg_uncorr.SetTitle(canvas_bkg_uncorr.GetTitle() + " " + title)  # Set title for upper plot
  label = ROOT.TPaveText(0.15, 0.25, 0.25, 0.35, "NDC")
  chi2Val = frame_bkg_uncorr.chiSquare()
  label.AddText(f"#chi^{{2}} = {chi2Val:.2f}")
  label.SetFillColor(0)
  label.SetBorderSize(0)
  frame_bkg_uncorr.addObject(label)
  canvas_bkg_uncorr.cd()
  frame_bkg_uncorr.Draw()

  ############### Draw correlated background spectrum with fit ###############
  canvas_bkg_corr = ROOT.TCanvas("canvas_bkg_corr" + title, "Background correlated", 800, 800)
  frame_bkg_corr = x.frame(RooFit.Bins(nBins))  # Apply binning
  data_mixingDeuteron.plotOn(frame_bkg_corr, RooFit.Binning(nBins))
  total_bkg_pdf.plotOn(frame_bkg_corr, RooFit.LineColor(kBlueC))
  total_bkg_pdf.paramOn(frame_bkg_corr, Layout = [0.9, 0.6, 0.9])
  if simBkgFit == True:
    total_bkg_pdf.plotOn(frame_bkg_corr, RooFit.Components("uncorr_bkg"), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(kOrangeC))
  # total_bkg_pdf.paramOn(frame_bkg_corr)
  frame_bkg_corr.GetYaxis().SetTitle(f'Counts / ({bin_width}' + ' GeV/#it{c}^{2})')
  frame_bkg_corr.GetXaxis().SetTitle('#it{M}(p+#pi+d) (GeV/#it{c}^{2})')
  frame_bkg_corr.SetTitle(canvas_bkg_corr.GetTitle() + " " + title)  # Set title for upper plot
  label = ROOT.TPaveText(0.15, 0.25, 0.25, 0.35, "NDC")
  chi2Val = frame_bkg_corr.chiSquare()
  label.AddText(f"#chi^{{2}} = {chi2Val:.2f}")
  label.SetFillColor(0)
  label.SetBorderSize(0)
  frame_bkg_corr.addObject(label)
  canvas_bkg_corr.cd()
  frame_bkg_corr.Draw()

  ############### Draw total fit ###############
  canvas_signal = ROOT.TCanvas("canvas_signal" + title, "SE Fit Results", 800, 1000)
  pad1 = ROOT.TPad("pad1", "Fit", 0, 0.3, 1, 1)  # Increase pad1 size
  pad2 = ROOT.TPad("pad2", "Residual", 0, 0, 1, 0.3)  # Decrease pad2 size
  pad1.SetBottomMargin(0.02)  # Reduce space between pads
  pad2.SetTopMargin(0)  # Remove top margin of lower pad
  pad2.SetBottomMargin(0.3)  # Increase space for x-axis
  pad1.Draw()
  pad2.Draw()

  # get number of signal and background
  signal_region_low = mu.getVal() - 3 * sigma.getVal()
  signal_region_high = mu.getVal() + 3 * sigma.getVal()
  x.setRange("signal", signal_region_low, signal_region_high)
  nSignal = frac_sig.getVal() * data_se.sumEntries()
  nSignal_err = nSignal * frac_sig.getError() / frac_sig.getVal()
  varBkg = total_bkg_pdf.createIntegral(ROOT.RooArgSet(x),ROOT.RooFit.NormSet(x),ROOT.RooFit.Range("signal"))
  expNBkg = data_se.sumEntries() * (1 - frac_sig.getVal()) * varBkg.getVal()
  unc_bkg = data_se.sumEntries() * varBkg.getVal() * frac_sig.getError()
  hRawYield = ROOT.TH1F("hRawYield", "hRawYield", 1, 0, 1)
  hRawYield.SetBinContent(1, nSignal)
  hRawYield.SetBinError(1, nSignal_err)

  # Avoid NaN
  if math.isnan(expNBkg):
    expNBkg = 0
  if math.isnan(unc_bkg):
    unc_bkg = 0
  # Get S/B
  if (expNBkg > 0):
    signal_to_bkg = nSignal / expNBkg
    signal_to_bkg_err = math.sqrt( math.pow(nSignal_err / expNBkg, 2) + math.pow( unc_bkg * nSignal / (expNBkg * expNBkg), 2) )
    signal_to_bkg_str = str(round(signal_to_bkg, 2)) + ' #pm ' + str(round(signal_to_bkg_err, 2))
  else:
    signal_to_bkg_str = 'NaN'
  
  # Get significance
  if (nSignal + expNBkg > 0):
    significance = nSignal / (math.sqrt(nSignal + expNBkg))
    significance_err = math.sqrt( math.pow( nSignal_err * (nSignal + 2*expNBkg) / (2 * (nSignal + expNBkg) * math.sqrt(nSignal + expNBkg)), 2) + math.pow( unc_bkg * nSignal / (2 * math.pow(nSignal + expNBkg, 1.5)), 2) )
    significance_str = str(round(significance, 2)) + ' #pm ' + str(round(significance_err, 2))
  else:
    significance_str = 'NaN'

  # Upper plot (full fit)
  pad1.cd()
  frame1 = x.frame(RooFit.Bins(nBins))  # Apply binning
  data_se.plotOn(frame1, RooFit.Binning(nBins), RooFit.Name("data"))  # Set binning for data
  total_pdf.plotOn(frame1, RooFit.Components("uncorr_bkg"), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(kGreenC))
  total_pdf.plotOn(frame1, RooFit.Components("total_sig_bkg_pdf"), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(kOrangeC), RooFit.Name("background"))
  total_pdf.plotOn(frame1, RooFit.LineColor(kBlueC))
  frame1.SetTitle(canvas_signal.GetTitle() + " " + title)  # Set title for upper plot
  frame1.GetXaxis().SetTitle("")  # Remove x-axis title in upper plot
  frame1.GetXaxis().SetLabelOffset(999)
  frame1.GetXaxis().SetLabelSize(0)
  frame1.GetYaxis().SetTitle(f'Counts / ({bin_width}' + ' GeV/c^{2})')
  frame1.GetYaxis().SetTitleSize(0.05)
  frame1.GetYaxis().SetLabelSize(0.04)
  frame1.GetYaxis().SetTitleOffset(1.2)
  frame1.Draw()
  # Info
  paveText = ROOT.TPaveText(0.14, 0.3, 0.7, 0.86, "NDC")
  paveText.SetName("paveText")
  paveText.SetBorderSize(0)
  paveText.SetFillStyle(0)
  paveText.SetTextFont(42)
  paveText.SetTextAlign(11)
  # paveText.SetTextSize(28)
  paveText.AddText('LHC24am, LHC24an, LHC24ao pass1')
  paveText.AddText('{0}'.format(ptlims[0]) + ' < #it{p}_{T} < ' + '{0}'.format(ptlims[1]) + ' GeV/#it{c}')
  paveText.AddText('')
  paveText.AddText('#mu = ' + str(round(mu.getValV(), 5)) + ' #pm ' + str(round(mu.getError(), 5)))
  paveText.AddText('#sigma = ' + str(round(sigma.getValV(), 5)) + ' #pm ' + str(round(sigma.getError(), 5)))
  paveText.AddText('')
  chi2Val = frame1.chiSquare()
  paveText.AddText(f"#chi^{{2}} = {chi2Val:.2f}")
  paveText.AddText('S = ' + str(round(nSignal)) + ' #pm ' + str(round(nSignal_err)))
  paveText.AddText('B(3#sigma) = ' + str(round(expNBkg)) + ' #pm ' + str(round(unc_bkg)))
  paveText.AddText('S/B(3#sigma) = ' + signal_to_bkg_str)
  paveText.AddText('Significance(3#sigma) = ' + significance_str)
  frame1.addObject(paveText)
  paveText.Draw()

  pad2.cd()
  residuals = frame1.residHist("data", "background")
  frame_residuals = x.frame(RooFit.Bins(nBins))
  frame_residuals.addPlotable(residuals, "PE")  # Add residuals properly
  signal.plotOn(frame_residuals, RooFit.LineColor(ROOT.kRed), RooFit.LineStyle(ROOT.kDashed), RooFit.Name("signal"))
  frame_residuals.SetTitle("")
  frame_residuals.GetXaxis().SetTitle('#it{M}(p+#pi+d) (GeV/#it{c}^{2})')
  frame_residuals.GetXaxis().SetTitleSize(0.1)
  frame_residuals.GetXaxis().SetLabelSize(0.1)
  frame_residuals.GetXaxis().SetTitleOffset(0.9)
  frame_residuals.GetYaxis().SetTitle("Residuals")
  frame_residuals.GetYaxis().SetTitleSize(0.1)
  frame_residuals.GetYaxis().SetLabelSize(0.08)
  frame_residuals.GetYaxis().SetTitleOffset(0.6)
  frame_residuals.SetMarkerStyle(20)
  frame_residuals.SetMarkerSize(0.8)
  frame_residuals.SetLineColor(ROOT.kBlack)
  frame_residuals.Draw()

  # Add dashed gray line at y=0
  line = ROOT.TLine(2.96, 0, 3.02, 0)
  line.SetLineColor(ROOT.kGray+2)
  line.SetLineStyle(2)
  line.SetLineWidth(2)
  pad2.cd()
  line.Draw()

  # Update canvas
  canvas_signal.Update()
  canvas_signal.RedrawAxis()

  return (hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal, nSignal, nSignal_err, expNBkg)

In [ ]:
############### Fit data ###############
signalCount = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0]), len(sigpdf), len(bkgpdf)] )
expBkgCount = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0]), len(sigpdf), len(bkgpdf)] )
signalError = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0]), len(sigpdf), len(bkgpdf)] )
if ifUseML:
    outfileName = "MLsimultaneousfit.root"
else:
    outfileName = "simultaneousfit.root"
with ROOT.TFile(savePrefix + outfileName, "recreate") as outfile:
    for icent, centbin in enumerate(CENT_BIN_LIST):
        savecentdir = outfile.mkdir("Cent"+str(centbin[0]) + "_" + str(centbin[1]))
        for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
            saveptdir = savecentdir.mkdir("pT"+str(ptbin[0]) + "_" + str(ptbin[1]))
            if ifUseML:
                for isig, sigfunc in enumerate(sigpdf):
                    savesigdir = saveptdir.mkdir(sigfunc)
                    for ibkg, bkgfunc in enumerate(bkgpdf):
                        savebkgdir = savesigdir.mkdir(bkgfunc)
                        for imodel, modelEff in enumerate(MODEL_Eff_LIST):
                            # Avoid warning due to low statistics and endless fit
                            if imodel < 10:
                                signalCount[icent][ipt][isig][ibkg].append(0)
                                signalError[icent][ipt][isig][ibkg].append(0.1)
                                expBkgCount[icent][ipt][isig][ibkg].append(999)
                                continue
                            binkey =  "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
                            binInfo = " pT " + str(ptbin[0]) + "-" + str(ptbin[1]) + "GeV/c " + " BDTEff=" +str(round(modelEff,2))
                            print("Fitting ", binInfo, " with ", sigfunc, " and ", bkgfunc)
                            if modelEff != 1:
                                model_threshold = score_eff_arrays_dict[binkey][0][imodel]
                                data_se = data.getDF(icent, ipt).query(f'model_output>{model_threshold}')
                                bkg_me_deuteron = bkg_mixed_deuteron.getDF(icent, ipt).query(f'model_output>{model_threshold}')
                                bkg_me_proton = bkg_mixed_proton.getDF(icent, ipt).query(f'model_output>{model_threshold}')
                            else:
                                data_se = data.getDF(icent, ipt)
                                bkg_me_deuteron = bkg_mixed_deuteron.getDF(icent, ipt)
                                bkg_me_proton = bkg_mixed_proton.getDF(icent, ipt)
                            # Avoid unexpected error
                            if len(data_se)==0:
                                signalCount[icent][ipt][isig][ibkg].append(0)
                                signalError[icent][ipt][isig][ibkg].append(0.1)
                                expBkgCount[icent][ipt][isig][ibkg].append(999)
                                continue
                            # (nsignal, err, expNBkg) = emTool.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_proton, savebkgdir, mcparas = MCFitpara[icent][ipt][0], title = binInfo, corr_bkgPdf = bkgfunc, uncorr_bkgPdf = "pol1")
                            # (hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal, nsignal, err, expNBkg) = emTool.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_proton, MCFitpara[icent][ipt][0], nBins = 26, ptlims = ptbin, lowMassLim = 2.975, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM")
                            # if ptbin[0] <= 2.6:
                            (hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal, nsignal, err, expNBkg) = simultaneousFit(data_se, bkg_me_deuteron, bkg_me_proton, MCFitpara[icent][ipt][0], nBins = 32, ptlims = ptbin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM")
                            savebkgdir.cd()
                            canvas_signal.Write()
                            canvas_bkg_corr.Write()
                            canvas_bkg_uncorr.Write()
                            del hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal
                            signalCount[icent][ipt][isig][ibkg].append(nsignal)
                            signalError[icent][ipt][isig][ibkg].append(err)
                            expBkgCount[icent][ipt][isig][ibkg].append(expNBkg)
            # else:
            #     binInfo = " pT " + str(ptbin[0]) + "-" + str(ptbin[1]) + "GeV/c "
            #     (nsignal, err, expNBkg) = emTool.simultaneousFit(data.getDF(icent, ipt), bkg_mixed_deuteron.getDF(icent, ipt), saveptdir, MCFitpara[icent][ipt][0], binInfo, extraBkgPdf="pol1", bkgPdf="DSCB", extraMixedBkgPdf="none")
            #     signalCount[icent][ipt].append(nsignal)
            #     signalError[icent][ipt].append(err)
            #     expBkgCount[icent][ipt].append(expNBkg)

In [ ]:
# Obtain Absorption factor
AbsorbFactor = fRead.getAbsorpFactor(CENT_BIN_LIST, PT_BIN_LIST, absorp_file = "../CC_file/absorption_histos_3b.root")
# AbsorbFactor = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] # To be implemented

############### Calculate efficiency and expected significance ###############
MCGenSignalCount = myH.createEmptyList( [len(CENT_BIN_LIST)] )
MCRecSignalCount = myH.createEmptyList( [len(CENT_BIN_LIST)] ) #reconstructed hypertriton count
ExpSignalYield = myH.createEmptyList( [len(CENT_BIN_LIST)] ) # expected number of reconstructed hypertriton via 3-body decay
Efficiency = myH.createEmptyList( [len(CENT_BIN_LIST)] )
ExpSignificance = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0]), len(sigpdf), len(bkgpdf)] )


with ROOT.TFile(savePrefix + "MLsimultaneousfit.root", "update") as outfile:
    for icent, centbin in enumerate(CENT_BIN_LIST):
        for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
            binCutMC = f'(fGenPt >= {ptbin[0]}) and (fGenPt < {ptbin[1]})'
            MCGenSignalCount[icent].append(len(MCPDRaw.query(f"fIsSignal == 1 and abs(fGenRapidity) < 0.5 and {binCutMC}")))
            MCRecSignalCount[icent].append(len(mcdata.dataPD[icent][ipt]))
            ExpSignalYield[icent].append( pt_spectrum.Integral(ptbin[0], ptbin[1]) * fRead.getEventNumber(dataType) * BR_3body)
            eff = MCRecSignalCount[icent][ipt]/MCGenSignalCount[icent][ipt] #efficiency with total recontructed hypertriton count
            Efficiency[icent].append(eff)
            print("Efficiency in ", f'( {centbin[0]} - {centbin[1]} ) and ( {ptbin[0]} <= pT < {ptbin[1]} ), :', eff)
            print("Expected Yield", pt_spectrum.Integral(ptbin[0], ptbin[1]) / (ptbin[1] - ptbin[0]) )
            print("Exp Raw SignalYield:", ExpSignalYield[icent][ipt] * eff * 2)

            binkey = "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
            # binkey = "ModelTest"
            for isig, sigfunc in enumerate(sigpdf):
                for ibkg, bkgfunc in enumerate(bkgpdf):
                    for imodel, modelEff in enumerate(MODEL_Eff_LIST):
                        corrfactor = eff * modelEff * AbsorbFactor[icent][ipt]
                        expsig = 2 * ExpSignalYield[icent][ipt] * corrfactor
                        if ifTellMatter:
                            expsig = expsig / 2
                        ExpSignificance[icent][ipt][isig][ibkg].append(expsig / math.sqrt(expsig + expBkgCount[icent][ipt][isig][ibkg][imodel]))

        #Plot efficiency versus pt in each centbin
        savecentdir = outfile.Get("Cent"+str(centbin[0]) + "_" + str(centbin[1]))
        c_efficiency = myH.TCanvas("efficiency"+"Cent"+str(centbin[0]) + "_" + str(centbin[1]),'efficiency')
        c_efficiency.cd()
        hMCGenSignalCount = ROOT.TH1F("hMCGenSignalCount", "hMCGenSignalCount", data.hist_ptBins[icent].size - 1, data.hist_ptBins[icent])
        hEff = ROOT.TH1F("hEff", ";#it{p}_{T} (GeV/#it{c});Precut Efficiency * Acceptance", data.hist_ptBins[icent].size - 1, data.hist_ptBins[icent])
        for ipt in range(data.hist_ptBins[icent].size - 1):
            hMCGenSignalCount.SetBinContent(ipt+1, MCGenSignalCount[icent][ipt])
            hMCGenSignalCount.SetBinError(ipt+1, 0)
            hEff.SetBinContent(ipt+1, MCRecSignalCount[icent][ipt])
        hEff.Divide(hMCGenSignalCount)
        hEff.GetYaxis().SetRangeUser(0, 0.08)
        hEff.SetMarkerStyle(8)
        hEff.SetMarkerSize(0.5)
        hEff.Draw("ep")
        c_efficiency.Write()

In [ ]:
############### QA and systematical uncertainties ###############
# FIXME: make code clearer
ModelIndex = myH.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0]), len(sigpdf)] )
SystUnc = myH.createEmptyList( [len(CENT_BIN_LIST)] )
with ROOT.TFile(savePrefix + "MLsimultaneousfit.root", "update") as outfile:
    for icent, centbin in enumerate(CENT_BIN_LIST):
        SystUnc.append([])
        savecentdir = outfile.Get("Cent"+str(centbin[0]) + "_" + str(centbin[1]))
        for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
            #BDT efficiency versus output of model
            saveptdir = savecentdir.Get("pT"+str(ptbin[0]) + "_" + str(ptbin[1]))
            #binkey = "Cent" + str(centbin[0]) + "_" + str(centbin[1]) + "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
            binkey = "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
            modelOutput_x = score_eff_arrays_dict[binkey][0][0:len(MODEL_Eff_LIST)]
            BDTefficiency_y = score_eff_arrays_dict[binkey][1][0:len(MODEL_Eff_LIST)]
            if len(MODEL_Eff_LIST) > len(BDTefficiency_y):
                BDTefficiency_y = np.append(BDTefficiency_y, 1)
            binInfo = "pT" + str(ptbin[0]) + "-" + str(ptbin[1]) + " GeV/c"
            c_BDTefficiency = myH.TCanvas()
            c_BDTefficiency.cd()
            h_Back_BDTeff = ROOT.TH2F("h_Back_BDTeff"+binInfo, binInfo + ";Model_cut;BDTefficiency", 1,-10,15, 1,0,1.1*np.max(BDTefficiency_y) )
            gr_BDTefficiency = ROOT.TGraph(len(MODEL_Eff_LIST), modelOutput_x, BDTefficiency_y)
            h_Back_BDTeff.Draw()
            gr_BDTefficiency.SetLineColor(4)
            gr_BDTefficiency.Draw("LP same")
            saveptdir.WriteObject(c_BDTefficiency, "BDTefficiency"+binkey)
            CorrectedSignal = [] #list of corrected signal to calculate systematic uncertainties
            for isig, sigfunc in enumerate(sigpdf):
                for ibkg, bkgfunc in enumerate(bkgpdf):
                    binInfo = "pT" + str(ptbin[0]) + "-" + str(ptbin[1]) + " GeV/c" + " " + sigfunc + " " + bkgfunc
                    significance = np.array(ExpSignificance[icent][ipt][isig][ibkg])
                    significanceMulBDTEff = significance * BDTefficiency_y
                    c_ModelSelection = myH.TCanvas()
                    c_ModelSelection.cd()
                    h_Back_modelsel = ROOT.TH2F("h_Back_modelsel" + binkey + sigfunc + bkgfunc, binInfo + ";Model_cut;Expected Significance #times BDT Efficiency", 1,-10,15, 1,0,1.1*np.max(significanceMulBDTEff) )
                    gr_ModelSelection = ROOT.TGraph(len(modelOutput_x), modelOutput_x, significanceMulBDTEff)
                    h_Back_modelsel.Draw()
                    gr_ModelSelection.SetLineColor(ROOT.kBlue)
                    gr_ModelSelection.Draw("L same")
                    saveptdir.WriteObject(c_ModelSelection, "ModelSelection" + binkey + sigfunc + bkgfunc)
                    
                    # Use the BDT threshold with the highest (significance * BDT efficiency) as the central value
                    maxindex=np.argmax(significanceMulBDTEff)
                    ModelIndex[icent][ipt][isig].append(maxindex)
                    # Corrected signal counts to BDT efficiency
                    c_SigToBDTefficiency = myH.TCanvas()
                    c_SigToBDTefficiency.cd()
                    SigToBDTefficiency_y = np.array(signalCount[icent][ipt][isig][ibkg])/BDTefficiency_y
                    SigToBDTefficiency_errory = np.array(signalError[icent][ipt][isig][ibkg])/BDTefficiency_y
                    gr_SigToBDTefficiency = ROOT.TGraphErrors(len(MODEL_Eff_LIST), np.array(MODEL_Eff_LIST), SigToBDTefficiency_y, np.zeros(len(MODEL_Eff_LIST)), SigToBDTefficiency_errory )
                    gr_SigToBDTefficiency1 = ROOT.TGraphErrors(len(MODEL_Eff_LIST), np.array(MODEL_Eff_LIST), SigToBDTefficiency_y)
                    tf1_sigtoBDTeff = ROOT.TF1("tf1_sigtoBDTeff","[0]",max(MODEL_Eff_LIST[0],0.01*maxindex),min(0.01*maxindex+0.2,MODEL_Eff_LIST[-1]))
                    tf1_sigtoBDTeff.SetParName(0,"mean")
                    tf1_sigtoBDTeff.SetLineColor(ROOT.kRed)
                    tf1_sigtoBDTeff.SetLineStyle(2)
                    tf1_sigtoBDTeff.SetLineWidth(2)
                    gr_SigToBDTefficiency.SetTitle(binInfo + ';Eff_{BDT};N_{Sig} / Eff_{BDT}')
                    gr_SigToBDTefficiency.SetFillColor(ROOT.kCyan-10)
                    gr_SigToBDTefficiency.Draw("3A")
                    gr_SigToBDTefficiency1.SetLineColor(4)
                    gr_SigToBDTefficiency1.SetMarkerStyle(8)
                    gr_SigToBDTefficiency1.SetMarkerSize(0.4)
                    # gr_SigToBDTefficiency1.Fit("tf1_sigtoBDTeff","R")
                    gr_SigToBDTefficiency1.Draw("same LP")
                    saveptdir.WriteObject(c_SigToBDTefficiency, "SigToBDTefficiency" + binkey + sigfunc + bkgfunc)
                    
                    # Vary BDT efficiency to calculate systematic uncertainties
                    for var in range(-10, 11):
                        if (maxindex + var >= len(MODEL_Eff_LIST)) or (maxindex + var < 0):
                            continue
                        CorrectedSignal.append( SigToBDTefficiency_y[maxindex+var]/(Efficiency[icent][ipt] * AbsorbFactor[icent][ipt]) )
            # Calculate systematic uncertainties
            binInfo = "pT" + str(ptbin[0]) + "-" + str(ptbin[1]) + " GeV/c"     
            # Use the last case of sig and bkg_mixed_deuteron func as central value to calculate the uncertainty (now we only use the rms)
            isig = len(sigpdf) - 1
            ibkg = len(bkgpdf) - 1
            
            CorrectedYield_y = np.array(CorrectedSignal)/(fRead.getEventNumber(dataType) * BR_3body * 2 * (ptbin[1] - ptbin[0]))
            if ifTellMatter:
                CorrectedYield_y = CorrectedYield_y / 2
            h_CorrectedYield = ROOT.TH1F("hCorrectedYield" + binkey, binInfo + ";Corrected Hypertriton yield;Counts", 20, 0.8*np.min(CorrectedYield_y), 1.2*np.max(CorrectedYield_y) )
            for sigVar in CorrectedYield_y:
                h_CorrectedYield.Fill(sigVar)
            saveptdir.WriteObject(h_CorrectedYield, "hCorrectedYield" + binkey)
            SystUnc[icent].append(h_CorrectedYield.GetStdDev())
            
            # New tree to store the corrected yield with 3 sigma region to central value
            mean = h_CorrectedYield.GetMean()
            rms  = h_CorrectedYield.GetRMS()
            xTTree = np.zeros(1, dtype=np.float64)
            h_CorrectedYieldTree = ROOT.TH1F("hCorrectedYieldTree" + binkey, binInfo + ";Corrected Hypertriton yield;Counts", 20, 0.8*np.min(CorrectedYield_y), 1.2*np.max(CorrectedYield_y) )
            CorrectedYieldTree = ROOT.TTree("CorrectedYield"+binkey, "CorrectedYield Tree")
            CorrectedYieldTree.Branch("CorrectedYield",xTTree,'CorrectedYield/D')
            for sigVar in CorrectedYield_y:
                if abs(sigVar - mean) <= 3 * rms:
                    xTTree[0] = sigVar
                    h_CorrectedYieldTree.Fill(sigVar)
                    CorrectedYieldTree.Fill()
            saveptdir.WriteObject(h_CorrectedYieldTree, "hCorrectedYieldTree" + binkey)
            saveptdir.WriteObject(CorrectedYieldTree, "CorrectedYield" + binkey)

In [ ]:
############### Calculate Yield ###############
pt_y =[]
for ipt, ptbin in enumerate(PT_BIN_LIST[0]):
    pt_y.append(ptbin[0])
pt_y.append(PT_BIN_LIST[0][-1][1])
hStat = ROOT.TH1F("hStat", ";#it{p}_{T} (GeV/c);R", len(pt_y) - 1, np.array(pt_y, dtype=np.float32))
hSyst = ROOT.TH1F("hSyst", ";#it{p}_{T} (GeV/c);R", len(pt_y) - 1, np.array(pt_y, dtype=np.float32))
# Use the last case of sig and bkg_mixed_deuteron func as central value
isig = len(sigpdf) - 1
ibkg = len(bkgpdf) - 1
for icent, centbin in enumerate(CENT_BIN_LIST):
    print("Cent " + str(centbin[0]) + "-" + str(centbin[1]) + "%")
    for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
        # binkey = "Cent" + str(centbin[0]) + "_" + str(centbin[1]) + "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
        binkey = "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
        BDTefficiency = score_eff_arrays_dict[binkey][1][0:len(MODEL_Eff_LIST)]
        maxindex = ModelIndex[icent][ipt][isig][ibkg]
        if len(MODEL_Eff_LIST) > len(BDTefficiency_y):
                BDTefficiency = np.append(BDTefficiency, 1)
        print(round(MODEL_Eff_LIST[maxindex],2), score_eff_arrays_dict[binkey][0][maxindex])
        
        corrfactor = BR_3body * fRead.getEventNumber(dataType) * Efficiency[icent][ipt] * BDTefficiency[maxindex] * AbsorbFactor[icent][ipt] * (ptbin[1] - ptbin[0])
        # Divide 2 if not distinguish between matter and antimatter
        if not ifTellMatter:
            corrfactor = corrfactor * 2
        hypyield  = signalCount[icent][ipt][isig][ibkg][maxindex] / corrfactor
        hypyield_error = signalError[icent][ipt][isig][ibkg][maxindex] / corrfactor
        # SystUnc[] already corrected

        hStat.SetBinContent(ipt+1, hypyield)
        hStat.SetBinError(ipt+1, hypyield_error)
        hSyst.SetBinContent(ipt+1, hypyield)
        hSyst.SetBinError(ipt+1, math.sqrt( math.pow( SystUnc[icent][ipt], 2) + math.pow( hypyield * (1 - AbsorbFactor[icent][ipt]) * 0.5, 2) ) )

In [ ]:
(rawyield_2body, statunc_2body, systunc_2body) = fRead.getH3L2bodyYield(PT_BIN_LIST)
h2bodyStat = ROOT.TH1F("h2bodyStat", ";#it{p}_{T} (GeV/c);R", len(pt_y) - 1, np.array(pt_y, dtype=np.float32))
h2bodySyst = ROOT.TH1F("h2bodySyst", ";#it{p}_{T} (GeV/c);R", len(pt_y) - 1, np.array(pt_y, dtype=np.float32))
for ipt, ptbin in enumerate(PT_BIN_LIST[0]):
    h2bodyStat.SetBinContent(ipt+1, rawyield_2body[ipt])
    h2bodyStat.SetBinError(ipt+1, statunc_2body[ipt])
    h2bodySyst.SetBinContent(ipt+1, rawyield_2body[ipt])
    h2bodySyst.SetBinError(ipt+1, systunc_2body[ipt])

In [ ]:
############### Save final results ###############
with ROOT.TFile(savePrefix + "Results.root", "recreate") as outfile:
    for icent, centbin in enumerate(CENT_BIN_LIST):
        for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
            maxindex = ModelIndex[icent][ipt][isig][ibkg]
            print(maxindex)
            # binkey = "ModelCent" + str(centbin[0]) + "_" + str(centbin[1]) + "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
            binkey =  "pT" + str(ptbin[0]) + "_" + str(ptbin[1])
            model_threshold = score_eff_arrays_dict[binkey][0][maxindex]
            binInfo = " pT " + str(ptbin[0]) + "-" + str(ptbin[1]) + "GeV/c " + " BDTEff=" +str(round(MODEL_Eff_LIST[maxindex],2)) #+ " " + dataType + " "
            data_se = data.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            bkg_me_deuteron = bkg_mixed_deuteron.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            bkg_me_proton = bkg_mixed_proton.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            # (nsignal, err, expNBkg) = emTool.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_proton, outfile, mcparas = MCFitpara[icent][ipt][0], title = binInfo, corr_bkgPdf = "dscb", uncorr_bkgPdf = "pol1")
            (hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal, nsignal, err, expNBkg) = simultaneousFit(data_se, bkg_me_deuteron, bkg_me_proton, MCFitpara[icent][ipt][0], nBins = 40, ptlims = ptbin, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol2", df_column="fM")
            canvas_signal.Write()
            canvas_bkg_corr.Write()
            canvas_bkg_uncorr.Write()
            del hRawYield, canvas_bkg_uncorr, canvas_bkg_corr, canvas_signal

        ############### Yield by assuming B.R. = 0.4 ###############
        ROOT.gStyle.SetOptStat(0)
        c_HypYields = myH.TCanvas('HypYields','HypYields')
        c_HypYields.cd()
        h_Back_Yields = ROOT.TH2F("h_Back_Yields", ";#it{p}_{T} (GeV/c);Yields", 1, 1.8, 5.2, 1, 0, 3.5*math.pow(10, -9))
        hStat.SetTitle("")
        hStat.GetXaxis().SetTitle( '#it{p}_{T} (GeV/c)' )
        hStat.GetYaxis().SetTitle( 'Yields' )
        hStat.SetMarkerColor(ROOT.kBlue)
        hStat.SetMarkerStyle(8)
        hStat.SetMarkerSize(1.5)
        hSyst.SetFillStyle(0)
        hSyst.SetLineColor(ROOT.kBlack)
        hSyst.SetMarkerColor(ROOT.kBlack)
        hSyst.SetMarkerStyle(20)
        h_Back_Yields.Draw("")
        hStat.Draw("Esamex0")
        hSyst.Draw("E2same")
        pt_spectrum.Draw("same")
        legend = ROOT.TLegend(0.6, 0.7, 0.9, 0.9)
        legend.AddEntry(pt_spectrum, "2body mTExpo fit", "l")
        legend.Draw()
        outfile.WriteObject(c_HypYields, 'HypYields')
        c_HypYields.SaveAs(savePrefix + "HypYields.pdf")

        ############### Ratio of B.R. ###############
        c_R = myH.TCanvas('c_R','c_R')
        c_R.cd()
        h_Ratio = ROOT.TH2F("h_Ratio", ";#it{p}_{T} (GeV/c);R", 1, 1.8, 5.2, 1, 0.1, 1)
        h_RStat = h2bodyStat.Clone("h_R")
        h_RSyst = h2bodySyst.Clone("h_RSyst")
        # Recorrect the BR of 3-body yield (2-body already corrected while readin)
        h_SumStat = hStat.Clone("h_SumStat")
        h_SumSyst = hSyst.Clone("h_SumSyst")
        h_SumStat.Scale(0.4)
        h_SumSyst.Scale(0.4)
        # Calculate the ratio
        h_SumStat.Add(h2bodyStat)
        h_SumSyst.Add(h2bodySyst)
        h_RStat.Divide(h_SumStat)
        h_RSyst.Divide(h_SumSyst)
        # Plot
        h_RStat.SetTitle("")
        h_RStat.GetXaxis().SetTitle( '#it{p}_{T} (GeV/c)' )
        h_RStat.GetYaxis().SetTitle( 'R' )
        h_RStat.SetLineColor(ROOT.kBlue)
        h_RStat.SetMarkerColor(ROOT.kBlue)
        h_RStat.SetMarkerStyle(8)
        h_RStat.SetMarkerSize(1.5)
        h_RSyst.SetFillStyle(1)
        h_RSyst.SetLineColor(ROOT.kBlack)
        h_RSyst.SetMarkerColor(ROOT.kBlack)
        h_RSyst.SetMarkerStyle(20)
        tf1_R = ROOT.TF1("tf1_R","[0]", 2, 5)
        h_RStat.Fit("tf1_R")
        h_Ratio.Draw("")
        h_RStat.Draw("Esamex0")
        h_RSyst.Draw("E2same")
        outfile.WriteObject(c_R, 'R')
        c_R.SaveAs(savePrefix + "R.pdf")